In [0]:
%py

import json

res = json.loads(spark.sparkContext.wholeTextFiles("/FileStore/tables/expert-1.json").collect()[0][1] \
  .replace("\"classifications\":\"none\"", "\"classifications\": []") \
  .replace("\"classifications\": \"none\"", "\"classifications\": []") \
  .replace("\"polygons\":\"none\"", "\"polygons\": []") \
  .replace("\"polygons\": \"none\"", "\"polygons\": []"))




dbutils.fs.put("/temp/pynormalized",  "\n".join(list(map(json.dumps, res))), True)

Wrote 1772232 bytes.
Out[3]: True

In [0]:
%scala

import org.apache.spark.sql.Encoders

val new_data = 
spark.read.json(
spark.read
  .textFile("/temp/pynormalized")
  .map(s=>{
    s.replaceAllLiterally("\"classifications\":\"none\"", "\"classifications\": []")
                      .replaceAllLiterally("\"polygons\":\"none\"", "\"polygons\": []")
  })
  )


new_data.createOrReplaceTempView("transoformed")

import org.apache.spark.sql.Encoders
new_data: org.apache.spark.sql.DataFrame = [Description: string, External ID: string ... 1 more field]

In [0]:
%scala
import org.apache.spark.sql.functions._
val df = new_data.select($"External ID".as("ext_id"), explode($"Label"("objects")).as("object"))
  .select($"ext_id", explode($"object"("classifications")).as("classification"))
  .select($"ext_id", $"classification"("value").as("level"), when($"classification"("answer").isNull, $"classification"("answers")).otherwise(array($"classification"("answer"))).as("answers") )
  .select($"ext_id", $"level", explode($"answers").as("answer"))
  .select($"ext_id", $"level", $"answer"("value").as("value"))
display(df)

ext_id,level,value
888.JPG,level_one,well_defined
888.JPG,level_two,radiopaque
888.JPG,level_three,none
888.JPG,level_four,developmental
888.JPG,level_three,none
888.JPG,level_four,developmental
149.JPG,level_one,well_defined
149.JPG,level_two,radiolucent
149.JPG,level_three,none
149.JPG,level_four,inflammation
